<a href="https://colab.research.google.com/github/hugogsmendes/Mini-Projetos-DSA/blob/main/Mini_Projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mini-Projeto 7**

## Deep Learning com PyTorch para Classificação de Imagens

### 1. Instalação e Importação dos Pacotes

https://pytorch.org

In [1]:
!pip install -q -U watermark

In [2]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from io import BytesIO
from torchsummary import summary

In [3]:
%reload_ext watermark
%watermark -a "Hugo Mendes" --iversions

Author: Hugo Mendes

PIL         : 11.3.0
matplotlib  : 3.10.0
numpy       : 2.0.2
torch       : 2.9.0+cu126
torchsummary: 1.5.1
torchvision : 0.24.0+cu126



### 2. Definindo o Dispositivo para Treinamento do Modelo de IA

In [4]:
# Bloco para seleção de dispositivos (CUDA, MPS ou CPU)

if torch.cuda.is_available():
  # Prioridade 1: GPU NVIDIA (CUDA)
  device = torch.device('cuda')
  print('Dispositivo selecionado: GPU NVIDIA (CUDA)')

elif torch.backends.mps.is_available():
  # Prioridade 2: GPU APPLE (MPS)
  device = torch.device('mps')
  print('Dispositivo selecionado: GPU Apple (MPS)')

else:
  # Prioridade 3: CPU
  device = torch.device('cpu')
  print('Dispositivo selecionado: CPU')

print(f'Usando dispositivo: {device}')

Dispositivo selecionado: GPU NVIDIA (CUDA)
Usando dispositivo: cuda


### 3. Definindo Hiperparâmetros para Treinamento do Modelo de IA

**num_epochs = 10** (Número de épocas)

*   Uma "época" representa uma passagem completa do algoritmo por todo o conjunto de dados de treinamento
*   O modelo "verá e aprenderá com cada exemplo de treinamento um total de 10 vezes

**batch_size = 64** (Tamanho do lote)
*   Em vez de mostrar ao modelo o conjunto de dados inteiro de uma só vez, nós dividimos em pequenos "lotes"
*   O modelo processará 64 exemplos de treinamento, calculará o erro médio desse lote e, em seguida, atualizará seus parâmetros (pesos) uma única vez. O processo de repete com os próximos 64 exemplos, e assim por diante, até que todos os dados da época tenham sido vistos

**learning_rate = 0.001** (Taxa de aprendizado)
*   Ele controla o "tamanho do passo" que o modelo dá ao ajustar seus pesos durante o treinamento
*   Após cada lote, o modelo calcula a direção para corrigir seus erros (o "gradiente"). Em vez de fazer uma correção drástica, ele fará uma correção pequena (multiplicada por 0.001) nessa direção

In [5]:
# Hiperparâmetros do modelo
num_epochs = 10 # Número de épocas para treinar
batch_size = 64 # Tamanho do lote (batch)
learning_rate = 0.001 # Taxa de aprendizado

### 4. Carregando os Dados (Imagens) e Definindo as Transformações, DataLoaders e Classes

*   transforms.ToTensor(): Converte a imagem (que tem valores de pixel de 0 a 255) em um Tensor do PyTorch e, o mais importante, ajusta a escala desses valores para um intervalo de [0.0, 1.0]
*   transforms.Normalize(): Pega esses valores [0.0, 1.0] e os centraliza, mudando o intervalo para [-1.0, 1.0]

In [6]:
# Definir as transformações para os dados
dsa_transformacoes = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [7]:
# Baixar e carregar o dataset de treino
dsa_dataset_treino = torchvision.datasets.CIFAR10(root='./data',
                                                  train=True,
                                                  download=True,
                                                  transform=dsa_transformacoes)

In [8]:
# Baixar e carregar o dataset de teste
dsa_dataset_teste = torchvision.datasets.CIFAR10(root='./data',
                                                 train=False,
                                                 download=True,
                                                 transform=dsa_transformacoes)

Precisamos dos DataLoaders para automatizar e otimizar a entrega dos dados ao modelo durante o treinamento e teste. Eles resolvem três problemas principais:

*   **Gerenciamento de memória (Lotes)**: Seu dataset (dsa_dataset_treino) pode ter 100.000 imagens. Elas não cabem todas na memória de uma só vez. O DataLoader pega esse dataset e o serve em "lotes" pequenos (batch_size = 64), um de cada vez
*   **Embaralhamento (Evitar Vício)**: shuffle=True (para treino) a cada época, ele embaralha os dados de treino. shuffle=False (para teste) garante que os dados de teste sejam avaliados na mesma ordem
*   **Eficiência (Paralelismo)**: O DataLoader pode usar múltiplos "trabalhadores" (processos) para carregar os próximos lotes de dados enquanto a GPU ainda está processando o lote atual, evitando gargalos e acelerando o treinamento

In [9]:
# Criar os DataLoaders para carregar os dados em lotes
dsa_loader_treino = torch.utils.data.DataLoader(dsa_dataset_treino,
                                                batch_size = batch_size,
                                                shuffle = True)

dsa_loader_teste = torch.utils.data.DataLoader(dsa_dataset_teste,
                                                batch_size = batch_size,
                                                shuffle = False)

In [10]:
# Definir as classes do CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [11]:
print(f'Número de imagens de treino: {len(dsa_dataset_treino)}')
print(f'Número de imagens de teste: {len(dsa_dataset_teste)}')
print(f'Número de lotes (batches) de treino: {len(dsa_loader_treino)}')
print(f'Número de lotes (batches) de teste: {len(dsa_loader_teste)}')

Número de imagens de treino: 50000
Número de imagens de teste: 10000
Número de lotes (batches) de treino: 782
Número de lotes (batches) de teste: 157
